### Connect and use Hopsworks Feature Store from Databricks

This notebook assumes that you have alreadly followed the instructions in the `Databricks-FeatureStore-Setup` notebook. In particular you should have already run the `setup_databricks` method and configured the cluster with the additional Spark Settings and Init Script.

Before being able to read/write into the Hopsworks Feature Store we need to connect to it. The connection parameters are the same of the `setup_databricks` method. As before, if you don't have access to the Secrets Manager or Parameter Store, you have the option of writing the API key on a file and pass the `api_key_file` parameter.

In [2]:
from  hops import featurestore

featurestore.connect(host="host.aws.hopsworks.ai", project_name="demo_featurestore_admin000", region_name="us-west-2", secrets_store="secretsmanager")

Once the connection has been established correctly we can start using the Hops Python API to interact with the Hopsworks Feature Store. You can find the API documentation at http://hops-py.logicalclocks.com. You can also run all the PySpark example you can find in our `hops-examples` Github repository: https://github.com/logicalclocks/hops-examples/tree/master/notebooks/featurestore - Just make sure you prepend the `featurestore.connect` method, before running any of the cells.

In the next cell we are going to demonstrate how you can use the Hops Python API to read an existing feature group and create a new one.

In [4]:
attendances_features = featurestore.get_featuregroup("attendances_features")

Running sql: use demo_featurestore_fabio000_featurestore against offline feature store
SQL string for the query created successfully
Running sql: SELECT * FROM attendances_features_1 against offline feature store

In [5]:
attendances_features.show(5)

+-------+------------------+--------------+
team_id|average_attendance|sum_attendance|
+-------+------------------+--------------+
 6| 19595.973| 391919.47|
 16| 6462.462| 129249.24|
 20| 7226.672| 144533.44|
 40| 3189.8455| 63796.91|
 9| 9405.213| 188104.27|
+-------+------------------+--------------+
only showing top 5 rows

For the sake of the example, let's add another column to the dataframe which will become the content for a new feature group.

In [7]:
from pyspark.sql.functions import rand
new_fg = attendances_features.withColumn("avg_age", rand() * 100)

Using the Hops Python API we can now write into the Hopsworks Feature Store the new Feature Group.

In [9]:
featurestore.create_featuregroup(
    new_fg,
    "attendances_features_databricks",
    description="Feature Group containing average attendances features created from Databricks",
    descriptive_statistics=True,
    feature_correlation=True,
    feature_histograms=True,
    cluster_analysis=True
)

--------------------------------------------------------------------------- 
 RestAPIError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/hops/featurestore.py in create_featuregroup (df, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 637 corr_method = corr_method , num_clusters = num_clusters , partition_by = partition_by , 
 --> 638 online=online, online_types=online_types, offline=offline)
 639 # If it fails, update cache and retry 

 /databricks/python/lib/python3.7/site-packages/hops/featurestore_impl/core.py in _do_create_featuregroup (df, featurestore_metadata, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 1851 num_bins , num_clusters , corr_method , featuregroup_type , featuregroup_type_dto , 
 -> 1852 None, None, online)
 1853 fs_utils . _log ( "Registering feature metadata... [COMPLETE]" ) 

 /databricks/python/lib/python3.7/site-packages/hops/featurestore_impl/rest/rest_rpc.py in _create_featuregroup_rest (featuregroup, featurestore_id, description, featuregroup_version, jobs, features_schema, feature_corr_data, featuregroup_desc_stats_data, features_histogram_data, cluster_analysis_data, feature_corr_enabled, featuregroup_desc_stats_enabled, features_histogram_enabled, cluster_analysis_enabled, stat_columns, num_bins, num_clusters, corr_method, featuregroup_type, featuregroup_dto_type, sql_query, jdbc_connector_id, online_fg) 
 290 "HTTP code: {}, HTTP reason: {}, error code: {}, error msg: {}, user msg: {}".format(
 --> 291 resource_url, response.status_code, response.reason, error_code, error_msg, user_msg))
 292 

 RestAPIError : Could not create feature group (url: /hopsworks-api/api/project/119/featurestores/67/featuregroups), server response: 
 HTTP code: 400, HTTP reason: Bad Request, error code: 270089, error msg: The feature group you are trying to create does already exist., user msg: project: demo_featurestore_fabio000, featurestoreId: 67

During handling of the above exception, another exception occurred:

 RestAPIError Traceback (most recent call last)
 <command-1218182466500379> in <module> 
 6 feature_correlation = True , 
 7 feature_histograms = True , 
 ----> 8 cluster_analysis = True 
 9 )

 /databricks/python/lib/python3.7/site-packages/hops/featurestore.py in create_featuregroup (df, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 646 cluster_analysis = cluster_analysis , stat_columns = stat_columns , num_bins = num_bins , 
 647 corr_method = corr_method , num_clusters = num_clusters , partition_by = partition_by , 
 --> 648 online=online, online_types=online_types, offline=offline)
 649 
 650 # update metadata cache since we created a new feature group and added new metadata 

 /databricks/python/lib/python3.7/site-packages/hops/featurestore_impl/core.py in _do_create_featuregroup (df, featurestore_metadata, featuregroup, primary_key, description, featurestore, featuregroup_version, jobs, descriptive_statistics, feature_correlation, feature_histograms, cluster_analysis, stat_columns, num_bins, corr_method, num_clusters, partition_by, online, online_types, offline) 
 1850 descriptive_statistics , feature_histograms , cluster_analysis , stat_columns , 
 1851 num_bins , num_clusters , corr_method , featuregroup_type , featuregroup_type_dto , 
 -> 1852 None, None, online)
 1853 fs_utils . _log ( "Registering feature metadata... [